In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import ease
import unittest

In [10]:
#define test data 
df = pd.read_csv('../Arranged_Data/final_weater.csv')[[
            'State', 'TotalMonthlyPrecip', 'TempSummer',
            'TempWinter', 'Avgwindspeed']]
df = df[df.State != 'DC']
test = pd.DataFrame()
#test sample size is 100
sample = np.random.randint(0,1050,(1,100))
for i in sample:
    test = test.append(df.iloc[i])


100

## Test rf

In [25]:
class TEST_RF(unittest.TestCase):
 
    def test_rf_input(self):
        prec = test.iloc[:,1]
        ts = test.iloc[:,2]
        tw = test.iloc[:,3]
        ws = test.iloc[:,4]
        #test the type of input dataset
        self.assertIsInstance(prec,pd.Series)
        self.assertIsInstance(ts,pd.Series)
        self.assertIsInstance(tw,pd.Series)
        self.assertIsInstance(ws,pd.Series)
        #test the length of input dataset
        self.assertEqual(len(prec), 100)
        self.assertEqual(len(ts), 100)
        self.assertEqual(len(tw), 100)
        self.assertEqual(len(ws), 100)
        #test the limitation of input dataset
        with self.assertRaises(IOError):
            ts > 100 or ts < tw or tw < 0
        with self.assertRaises(IOError):
            prec < 0 # only test lower limit
        with self.assertRaises(IOError):
            ws < 0 # only test lower limit

    def test_rf_output(self):
        prec = test.iloc[:,1]
        ts = test.iloc[:,2]
        tw = test.iloc[:,3]
        ws = test.iloc[:,4]
        output = ease.rf(prec, ts, tw, ws)
        #test the type of output dataset
        self.assertIsInstance(output,dict)
        #test the output values
        with self.assertRaises(ValueError):
            for i in list(output.values()):
                0 < i < 1


## Test avg_capacity

In [29]:
def avg_capacity(vote):
    average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
    avg_cap_list = []
    coal_sum = 0
    ng_sum = 0
    petro_sum = 0
    hydro_sum = 0
    solar_sum = 0
    wind_sum =0
    for i in vote.keys():
        coal_sum += int(average_plant_capacity.Coal[average_plant_capacity.State == i]) * vote[i]
        ng_sum += int(average_plant_capacity.NG[average_plant_capacity.State == i]) * vote[i]
        petro_sum += int(average_plant_capacity.Petro[average_plant_capacity.State == i]) * vote[i]
        hydro_sum += int(average_plant_capacity.Hydro[average_plant_capacity.State == i]) * vote[i]
        solar_sum += int(average_plant_capacity.Solar[average_plant_capacity.State == i]) * vote[i]
        wind_sum += int(average_plant_capacity.Wind[average_plant_capacity.State == i]) * vote[i]
    return([coal_sum, ng_sum, petro_sum, hydro_sum, solar_sum, wind_sum])

In [18]:
vote = rf(3, 67, 30, 4)

In [34]:
len(avg_capacity(vote))

6

In [35]:
class TEST_avg_capacity(unittest.TestCase):
    average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
    def test_avg_cap_input(self):
        prec = test.iloc[:,1]
        ts = test.iloc[:,2]
        tw = test.iloc[:,3]
        ws = test.iloc[:,4]
        vote = rf(prec, ts, tw, ws)
        output = avg_capacity(vote)
        #test the type of output dataset
        self.assertIsInstance(output,list)
        #test the length of output dataset
        self.assertEqual(len(output),6) #output list length is 6